In [ ]:
%pip install langchain_groq langchain_core langchain_community pypdf chromadb sentence_transformers gradio tf_keras ipywidgets vaderSentiment gradio

In [4]:
import os
import json
from collections import deque
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langchain_groq import ChatGroq
import datetime
import random
import gradio as gr

# -------------------------------
# Helper Functions & Global Setup
# -------------------------------

# File to store user data
USER_DATA_FILE = "user_data.json"

# University mental health resources
UNIVERSITY_RESOURCES = {
    "Centennial College": "Visit the Student Wellness Centre: https://www.centennialcollege.ca/student-health",
    "University of Toronto": "Check U of T’s mental health services: https://mentalhealth.utoronto.ca/",
}

# Motivational quotes for students
MOTIVATIONAL_QUOTES = [
    "Stay focused! Every small step brings you closer to success. 💪",
    "You’re capable of amazing things—keep pushing forward!",
    "Don’t let stress take over! Take breaks, breathe, and keep going. 🚀"
]

def load_user_data():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as file:
            return json.load(file)
    return {}

def save_user_data(user_data):
    with open(USER_DATA_FILE, "w") as file:
        json.dump(user_data, file, indent=4)

def update_user_data(user_id, key, value):
    user_data = load_user_data()
    if user_id not in user_data:
        user_data[user_id] = {}
    user_data[user_id][key] = value
    save_user_data(user_data)

def update_student_profile(user_id, major, year_of_study, common_stressors, university):
    user_data = load_user_data()
    if user_id not in user_data:
        user_data[user_id] = {}
    user_data[user_id]["major"] = major
    user_data[user_id]["year_of_study"] = year_of_study
    user_data[user_id]["common_stressors"] = common_stressors
    user_data[user_id]["university"] = university
    save_user_data(user_data)

def get_mental_health_resources(user_id):
    user_data = load_user_data()
    university = user_data.get(user_id, {}).get("university", "your university")
    if university in UNIVERSITY_RESOURCES:
        return f"If you need support, check out {UNIVERSITY_RESOURCES[university]}"
    else:
        return "I recommend checking your university's website for student wellness resources."

def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores["compound"]
    if compound_score <= -0.5:
        return "sad"
    elif -0.5 < compound_score <= -0.1:
        return "frustrated"
    elif -0.1 < compound_score < 0.1:
        return "neutral"
    elif 0.1 <= compound_score < 0.5:
        return "happy"
    else:
        return "excited"

def load_llm():
    return ChatGroq(
        temperature=0.2,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

def generate_summary(conversation_history, llm):
    prompt = f"""
    You are a helpful assistant. Summarize this conversation in 1-2 sentences.
    
    Conversation:
    {" ".join(conversation_history)}
    
    Summary:
    """
    response = llm.invoke(prompt)
    return response.content.strip()

def check_deadlines(user_id):
    user_data = load_user_data()
    deadlines = user_data.get(user_id, {}).get("deadlines", {})
    today = datetime.date.today()
    upcoming = [task for task, date in deadlines.items() if datetime.date.fromisoformat(date) <= today + datetime.timedelta(days=3)]
    if upcoming:
        return f"Reminder! You have upcoming deadlines: {', '.join(upcoming)}. Don’t forget to plan ahead!"
    return "No major deadlines soon. Keep up the good work!"

def send_daily_motivation():
    return random.choice(MOTIVATIONAL_QUOTES)

# -----------------------------------------
# Chatbot Function for Gradio Interface
# -----------------------------------------

def chatbot_response(user_message, history, user_id):
    """
    This function takes the latest user message, the chat history,
    and the user_id (name), then builds a prompt for the LLM and returns
    the updated conversation history.
    """
    # Initialize the LLM (you may want to optimize by caching it)
    llm = load_llm()
    
    # Load any stored user data (for example, major or previous emotions)
    user_data = load_user_data()
    major = user_data.get(user_id, {}).get("major", "student")
    
    # Rebuild conversation history as text
    conversation_history = []
    for pair in history:
        conversation_history.append(f"You: {pair[0]}")
        conversation_history.append(f"Chatbot: {pair[1]}")
    conversation_history.append(f"You: {user_message}")
    
    # Build the prompt with additional student details if available
    prompt = f"""
    You are a mental health assistant for students. The user is studying {major}.
    - Name: {user_id}
    - Last emotion: {user_data.get(user_id, {}).get("last_emotion", None)}
    - Last conversation: {user_data.get(user_id, {}).get("last_conversation", None)}
    
    {chr(10).join(conversation_history)}
    Assistant:
    """
    response = llm.invoke(prompt)
    bot_reply = response.content.strip()
    
    history.append((user_message, bot_reply))
    return history, history

# ----------------------------
# Gradio Blocks UI Definition
# ----------------------------

with gr.Blocks() as demo:
    gr.Markdown("# Mental Health Chatbot")
    
    with gr.Tab("User Setup"):
        gr.Markdown("### Set up your profile")
        user_name = gr.Textbox(label="Enter your name")
        major = gr.Textbox(label="Your major")
        year_of_study = gr.Textbox(label="Year of Study")
        common_stressors = gr.Textbox(label="Common stressors")
        university = gr.Textbox(label="University")
        setup_button = gr.Button("Set Up Profile")
        setup_output = gr.Textbox(label="Setup Status", interactive=False)
    
    with gr.Tab("Chat"):
        gr.Markdown("### Chat with the Bot")
        chatbot = gr.Chatbot(label="Conversation", type="messages")
        msg = gr.Textbox(label="Your Message")
        clear = gr.Button("Clear Chat")
    
    # Gradio States to hold conversation history and the current user id
    state = gr.State([])
    user_id_state = gr.State("")
    
    def setup_profile(name, major_val, year, stressors, univ):
        """
        Save the profile to our JSON file and update the user_id state.
        """
        if name.strip() == "":
            return "Please enter a valid name.", ""
        update_user_data(name, "name", name)
        update_student_profile(name, major_val, year, stressors, univ)
        status = f"Profile set up for {name}. Welcome!"
        return status, name

    setup_button.click(
        setup_profile, 
        inputs=[user_name, major, year_of_study, common_stressors, university], 
        outputs=[setup_output, user_id_state]
    )
    
    def respond(message, chat_history, user_id):
        return chatbot_response(message, chat_history, user_id)
    
    msg.submit(respond, [msg, state, user_id_state], [chatbot, state])
    clear.click(lambda: ([], []), None, [chatbot, state])

# Launch the Gradio demo!
demo.launch()


C:\Users\gcast\AppData\Local\Temp\ipykernel_34396\1591899095.py:170: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
